In [19]:
state_fips = ca.FIPS.apply(lambda x: x[:2]).unique().tolist()[0]

In [21]:
stateDf.dtypes

State          object
FIPS2          object
StateAbbrev    object
dtype: object

In [14]:
location = stateDf.loc[stateDf.FIPS2.isin(state_fips), 'StateAbbrev'].values.tolist()

,State,FIPS2,StateAbbrev
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA
5,Colorado,08,CO
6,Connecticut,09,CT
7,Delaware,10,DE
8,District of Columbia,11,DC
9,Florida,12,FL


In [1]:
import pickle
import pandas as pd

In [2]:
data_file_path = 'cif_raw_data.pickle'
with open(data_file_path, 'rb') as f:
    data_dictionary = pickle.load(f)

ca_file_path = './uky_ca.csv'
ca = pd.read_csv(ca_file_path, dtype={'FIPS':str})
ca['County'] = ca.County + ' County'

from utils import stateDf

In [3]:
def select_area_for_catchment_area_full(df, query_level, ca):
    if hasattr(df, 'County'):
        if df.County[0][-6:].lower() != 'county':
            df['County'] = df.County + ' County'
    
    if query_level == 'county':
        return df.loc[df.FIPS.isin(ca.FIPS), :].reset_index(drop = True)
    elif query_level in ['county subdivision','tract','block']:
        if hasattr(df, 'FIPS5'):
            df = df.loc[df.FIPS5.isin(ca.FIPS), :].reset_index(drop = True)
            df.drop('FIPS5', axis = 1, inplace = True)
        else:
            df = df.loc[df.FIPS.apply(lambda x: x[:5]).isin(ca.FIPS), :].reset_index(drop = True)
        return df


In [4]:
from functools import partial

In [5]:
select_area_for_catchment_area = partial(select_area_for_catchment_area_full, ca = ca)


In [6]:
rfs_tract = select_area_for_catchment_area(data_dictionary['tract']['risk_and_screening'], 'tract')    


In [8]:
rfs_tract

,FIPS,County,State,Cancer_Prevalence,Met_Colon_Screen,Currently_Smoke,Met_Breast_Screen,BMI_Obese
0,21049020500,Clark County,Kentucky,7.3,76.1,18.1,76.2,33.3
1,21101020500,Henderson County,Kentucky,6.7,70.5,24.2,73.2,42.1
2,21173920100,Montgomery County,Kentucky,6.3,72.8,22.3,72.5,37.5
3,21111011106,Jefferson County,Kentucky,6.3,74.1,19.6,75.4,35.7
4,21227011002,Warren County,Kentucky,3.4,67.8,24.0,76.1,38.8
...,...,...,...,...,...,...,...,...
1101,21067001400,Fayette County,Kentucky,5.6,64.4,33.3,70.8,41.8
1102,21097950500,Harrison County,Kentucky,8.1,69.5,24.2,70.7,38.6
1103,21111010318,Jefferson County,Kentucky,7.8,79.2,10.3,77.6,29.0
1104,21199930502,Pulaski County,Kentucky,6.9,68.9,26.8,71.3,42.4


In [7]:
    rfs_tract_l = pd.melt(rfs_tract, id_vars=['FIPS', 'Tract', 'County', 'State'], 
                         var_name='measure', value_name='value')


KeyError: "The following 'id_vars' are not present in the DataFrame: ['Tract']"

#### places data

In [4]:
def gen_long_risk_and_screening(data_dictionary = data_dictionary):
    df_county = data_dictionary['county']['risk_and_screening'].copy()
    df_tract  = data_dictionary['tract']['risk_and_screening'].copy()
    
    df_county = select_area_for_catchment_area(df_county, 'county')
    df_tract = select_area_for_catchment_area(df_county, 'tract')
    
    df_county_l = pd.melt(df_county, id_vars=['FIPS', 'County', 'State'], 
                         var_name='measure', value_name='value')

    df_tract_l = pd.melt(df_tract, id_vars=['FIPS', 'County', 'State'], 
                         var_name='measure', value_name='value')

    return df_county_l, df_tract_l
    
    

In [184]:
rfs_county, rfs_tract     = data_dictionary['county']['risk_and_screening'], data_dictionary['tract']['risk_and_screening']

In [185]:
rfs_county_l, rfs_tract_l = gen_long_risk_and_screening()

In [187]:
df_county = select_area_for_catchment_area(rfs_county, 'county')

In [188]:
df_county

,FIPS,County,State,Cancer_Prevalence,Met_Cervical_Screen,Met_Colon_Screen,Currently_Smoke,Met_Breast_Screen,BMI_Obese
0,21085,Grayson County,Kentucky,7.5,78.3,70.9,27.1,64.3,41.5
1,21061,Edmonson County,Kentucky,7.8,79.3,72.0,25.5,64.7,40.7
2,21223,Trimble County,Kentucky,7.6,80.1,71.3,23.6,67.3,38.7
3,21075,Fulton County,Kentucky,7.7,79.0,72.3,26.4,67.8,42.3
4,21179,Nelson County,Kentucky,7.1,82.3,75.3,19.5,70.0,37.9
...,...,...,...,...,...,...,...,...,...
115,21031,Butler County,Kentucky,7.6,79.7,72.0,24.8,66.9,39.2
116,21087,Green County,Kentucky,8.1,78.0,69.2,27.6,65.0,42.1
117,21189,Owsley County,Kentucky,8.0,77.7,69.0,28.8,65.4,41.5
118,21151,Madison County,Kentucky,6.2,80.8,74.7,19.3,71.1,34.0


#### gen_cancer_data

In [40]:
cancer_inc_l = data_dictionary['cancer']['incidence'].copy().convert_dtypes()
cancer_inc_l = select_area_for_catchment_area(cancer_inc_l, 'county')
cancer_inc_l = cancer_inc_l[['FIPS', 'County', 'State', 'Type', 'Site', 'AAR', 'AAC']]
cancer_inc = pd.pivot(cancer_inc_l, index=['FIPS', 'County', 'State', 'Type'], columns='Site', values='AAR').reset_index()
cancer_mor_l = data_dictionary['cancer']['mortality'].copy().convert_dtypes()
cancer_mor_l = select_area_for_catchment_area(cancer_mor_l, 'county')
cancer_mor_l = cancer_mor_l[['FIPS', 'County', 'State', 'Type', 'Site', 'AAR', 'AAC']]
cancer_mor = pd.pivot(cancer_mor_l, index=['FIPS', 'County', 'State', 'Type'], columns='Site', values='AAR').reset_index()

#### econ_data

In [46]:
def merge_all(*args, query_level = 'county'):
    if query_level == 'county':
        geo_col = ['FIPS','County','State']
    elif query_level == 'tract':
        geo_col = ['FIPS','Tract','County','State']
    datasets_to_merge = []
    datasets_not_to_merge = []
    columns_to_be_used_later = []
    for df in args:
        if all([hasattr(df, col) for col in geo_col]):
            datasets_to_merge.append(df)
        else:
            datasets_not_to_merge.append(df)
            columns_to_be_used_later.append(pd.Series(geo_col)[[hasattr(df, col) for col in geo_col]].tolist())
    for i, df in enumerate(datasets_to_merge):
        if i == 0:
            for col in geo_col:
                assert hasattr(df, col)
            output = df.copy()
        else:
            output = output.merge(df, how = 'left', on = geo_col)
    if len(datasets_not_to_merge):
        for df, merge_on_col in zip(datasets_not_to_merge, columns_to_be_used_later):
            output = output.merge(df, how = 'left', on = merge_on_col)
    return output

In [69]:
def organize_table(topic_variables, query_level, column_names_dict = None, columns_to_drop = None):
    topic_dataset = [select_area_for_catchment_area(data_dictionary[query_level][topic], query_level) for topic in topic_variables if topic in data_dictionary[query_level].keys()]
    df = merge_all(*topic_dataset, query_level = query_level)
    if column_names_dict:
        df = df.rename(columns = colnames)
    if columns_to_drop:
        df = df.drop(columns = columns_to_drop, axis = 1)
    return df.convert_dtypes()

In [105]:
econ_topics = ['insurance','gini_index','income','employment','poverty','bls_unemployment']

In [106]:
colnames = {'Labor Force Participation Rate': f'Annual Labor Force Participation Rate (2015-2019)',
            'Unemployment Rate' : f'Annual Unemployment Rate (2015-2019)',
            'health_insurance_coverage_rate': 'Insurance Coverage',
            'Gini Index': 'Gini Coefficient',
            'median_income_all': 'Household Income',
            'medicaid' : 'Medicaid Enrollment',
            'below_poverty' : 'Below Poverty'
            }

In [107]:
drop_col = ['below_poverty_x.5', 'below_poverty_x2']

In [108]:
econ_county = organize_table(econ_topics, 'county', colnames, drop_col)
econ_tract  = organize_table(econ_topics, 'tract', colnames, drop_col)
econ_county['Uninsured'] = 1 - econ_county['Insurance Coverage']
econ_tract['Uninsured'] = 1 - econ_tract['Insurance Coverage']

In [109]:
econ_county_l = pd.melt(econ_county, id_vars = ['FIPS', 'County', 'State'], 
                        var_name = 'measure', value_name = 'value')
econ_tract_l = pd.melt(econ_tract, id_vars = ['FIPS', 'County', 'Tract','State'], 
                        var_name = 'measure', value_name = 'value')

#### housing and transportation

In [111]:
ht_topic = ['vacancy','transportation']

In [112]:
colnames = {'vacancy_rate': 'Vacancy Rate', 'no_vehicle': 'No Vehicle',
                                         'rent_over_40':'Rent Burden (40% Income)'}

In [113]:
cols_to_drop = ['two_or_more_vehicle','three_or_more_vehicle']

In [114]:
ht_county = organize_table(ht_topic, 'county', colnames, cols_to_drop)
ht_tract = organize_table(ht_topic, 'tract', colnames, cols_to_drop)


In [115]:
ht_county_l = pd.melt(ht_county, id_vars = ['FIPS', 'County', 'State'], 
                        var_name = 'measure', value_name = 'value')
ht_tract_l = pd.melt(ht_tract, id_vars = ['FIPS', 'County','Tract', 'State'], 
                        var_name = 'measure', value_name = 'value')

#### Sociodemographic

In [116]:
socio_topic = ['demographic_age','demographic_race','education','urban_rural']

In [168]:
sociodemo_county = organize_table(socio_topic, 'county')
sociodemo_tract = organize_table(socio_topic, 'tract')
sd_county_l = pd.melt(sociodemo_county, id_vars = ['FIPS', 'County', 'State'], 
                        var_name = 'measure', value_name = 'value')
sd_tract_l = pd.melt(sociodemo_tract, id_vars = ['FIPS', 'County', 'Tract', 'State'], 
                        var_name = 'measure', value_name = 'value')


#### env_data

In [122]:
env_topic = ['water_violation','food_desert']

In [123]:
data_dictionary['county']['water_violation'] = \
data_dictionary['county']['vacancy'].merge(data_dictionary['county']['water_violation'], on = ['County','State']).sort_values('FIPS').reset_index(drop = True)

In [126]:
data_dictionary['county']['water_violation'] = \
data_dictionary['county']['water_violation'].drop('vacancy_rate', axis = 1)

In [130]:
data_dictionary['tract']['food_desert'] = \
    data_dictionary['tract']['vacancy'].merge(data_dictionary['tract']['food_desert']).drop(['vacancy_rate'],axis = 1)

In [139]:
env_county = organize_table(env_topic, 'county')
env_tract = organize_table(env_topic, 'tract')
env_county_l = pd.melt(env_county, id_vars = ['FIPS', 'County', 'State'], 
                        var_name = 'measure', value_name = 'value')
env_tract_l = pd.melt(env_tract, id_vars = ['FIPS', 'County', 'Tract', 'State'], 
                        var_name = 'measure', value_name = 'value')


#### facilities

In [151]:
superfund = data_dictionary['facility']['superfund']

In [153]:
superfund = select_area_for_catchment_area(superfund, 'tract')

In [155]:
superfund = superfund.convert_dtypes()

In [170]:
point_df = pd.concat([data_dictionary['facility']['all'], superfund], axis = 0).sort_values('Type').reset_index(drop = True)

#### Saving

In [171]:
cdata = {'rf_and_screening_county': rfs_county, 'rf_and_screening_county_long': rfs_county_l,
                'rf_and_screening_tract': rfs_tract, 'rf_and_screening_tract_long': rfs_tract_l,
                'cancer_incidence': cancer_inc, 'cancer_incidence_long': cancer_inc_l,
                'cancer_mortality': cancer_mor, 'cancer_mortality_long': cancer_mor_l,
                'economy_county': econ_county, 'economy_county_long': econ_county_l,
                'economy_tract': econ_tract, 'economy_tract_long': econ_tract_l,
                'ht_county': ht_county, 'ht_county_long': ht_county_l, 
                'ht_tract': ht_tract, 'ht_tract_long': ht_tract_l, 
                'sociodemographics_county': sociodemo_county, 'sd_county_long': sd_county_l,
                'sociodemographics_tract': sociodemo_tract, 'sd_tract_long': sd_tract_l,
                'environment_county': env_county, 'environment_county_long': env_county_l,
                'environment_tract': env_tract, 'environment_tract_long': env_tract_l,
                 'facilities_and_providers': point_df}

In [179]:
from glob import glob
glob('./CIFTools_Documentation.csv')

[]

In [183]:
cdata.keys()

dict_keys(['rf_and_screening_county', 'rf_and_screening_county_long', 'rf_and_screening_tract', 'rf_and_screening_tract_long', 'cancer_incidence', 'cancer_incidence_long', 'cancer_mortality', 'cancer_mortality_long', 'economy_county', 'economy_county_long', 'economy_tract', 'economy_tract_long', 'ht_county', 'ht_county_long', 'ht_tract', 'ht_tract_long', 'sociodemographics_county', 'sd_county_long', 'sociodemographics_tract', 'sd_tract_long', 'environment_county', 'environment_county_long', 'environment_tract', 'environment_tract_long', 'facilities_and_providers'])

In [181]:
from pandas import ExcelWriter
from datetime import datetime as dt
import os

ca_dir = 'Markey' + "_catchment_data"
path2 = os.path.join(os.getcwd(), ca_dir)

if os.path.exists(path2) == False:
    os.makedirs(path2)

save_name = ca_dir + '_' + dt.today().strftime('%m-%d-%Y') + '.xlsx'
save_name2 = ca_dir + '_long_' + dt.today().strftime('%m-%d-%Y') + '.xlsx'
full_path = os.path.join(os.getcwd(), ca_dir, save_name)
full_path2 = os.path.join(os.getcwd(), ca_dir, save_name2)

def write_excel_file(cdata, full_path, full_path2):
    
    cdata_keys = ['rf_and_screening_county', 'rf_and_screening_county_long', 
                  'rf_and_screening_tract', 'rf_and_screening_tract_long', 
                  'cancer_incidence', 'cancer_incidence_long', 
                  'cancer_mortality', 'cancer_mortality_long', 
                  'economy_county', 'economy_county_long', 
                  'economy_tract', 'economy_tract_long', 
                  'ht_county', 'ht_county_long', 'ht_tract', 'ht_tract_long', 
                  'sociodemographics_county', 'sd_county_long', 
                  'sociodemographics_tract', 'sd_tract_long', 
                  'environment_county', 'environment_county_long', 
                  'environment_tract', 'environment_tract_long', 
                  'facilities_and_providers']
    
    for name in cdata.keys():
        assert name in cdata_keys
    
    with ExcelWriter(full_path, mode = 'w') as writer:
        print('Writing wide data to file...')
        pd.read_csv('CIFTools_Documentation.csv', 
                    header = None, encoding = "ISO-8859-1").to_excel(writer, header = None, 
                                                                     sheet_name = 'Variables and Sources', index = False)
        cdata['cancer_incidence'].to_excel(writer, sheet_name = 'Cancer Incidence', index = True)
        cdata['cancer_mortality'].to_excel(writer, sheet_name = 'Cancer Mortality', index = True)
        cdata['economy_county'].to_excel(writer, sheet_name = 'Economy (County)', index = False)
        cdata['economy_tract'].to_excel(writer, sheet_name = 'Economy (Tract)', index = False)
        cdata['environment_county'].to_excel(writer, sheet_name = 'Environment (County)', index = False)
        cdata['environment_tract'].to_excel(writer, sheet_name = 'Environment (Tract)', index = False)
        #cdata['broadband_speeds'].to_excel(writer, sheet_name = 'Broadband Speeds', index = False) #can be too long in some areas
        cdata['ht_county'].to_excel(writer, sheet_name = 'H and T (County)', index = False)
        cdata['ht_tract'].to_excel(writer, sheet_name= 'H and T (Tract)', index = False)
        cdata['rf_and_screening_county'].to_excel(writer, sheet_name= 'RF and Screening (County)', index=True)
        cdata['rf_and_screening_tract'].to_excel(writer, sheet_name= 'RF and Screening (Tract)', index=True)
        cdata['sociodemographics_county'].to_excel(writer, sheet_name = 'Sociodemographic (County)', index = False)
        cdata['sociodemographics_tract'].to_excel(writer, sheet_name = 'Sociodemographic (Tract)', index = False)
        cdata['facilities_and_providers'].to_excel(writer, sheet_name = 'Facilities', index = False)

    with ExcelWriter(full_path2, mode = 'w') as writer:
        print('Writing long data to file...')
        pd.read_csv('CIFTools_Documentation.csv', 
                    header = None, encoding = "ISO-8859-1").to_excel(writer, header = None, 
                                                                     sheet_name = 'Variables and Sources', index = False)
        cdata['cancer_incidence_long'].to_excel(writer, sheet_name = 'Cancer Incidence', index = True)
        cdata['cancer_mortality_long'].to_excel(writer, sheet_name = 'Cancer Mortality', index = True)
        cdata['economy_county_long'].to_excel(writer, sheet_name = 'Economy (County)', index = False)
        cdata['economy_tract_long'].to_excel(writer, sheet_name = 'Economy (Tract)', index = False)
        cdata['environment_county_long'].to_excel(writer, sheet_name = 'Environment (County)', index = False)
        cdata['environment_tract_long'].to_excel(writer, sheet_name = 'Environment (Tract)', index = False)
        #cdata['broadband_speeds'].to_excel(writer, sheet_name = 'Broadband Speeds', index = False) # can be too long in some areas
        cdata['ht_county_long'].to_excel(writer, sheet_name = 'H and T (County)', index = False)
        cdata['ht_tract_long'].to_excel(writer, sheet_name= 'H and T (Tract)', index = False)
        cdata['rf_and_screening_county_long'].to_excel(writer, sheet_name= 'RF and Screening (County)', 
                                                        index=True)
        cdata['rf_and_screening_tract_long'].to_excel(writer, sheet_name= 'RF and Screening (Tract)', 
                                                      index=True)
        cdata['sd_county_long'].to_excel(writer, sheet_name = 'Sociodemographic (County)', index = False)
        cdata['sd_tract_long'].to_excel(writer, sheet_name = 'Sociodemographic (Tract)', index = False)
        cdata['facilities_and_providers'].to_excel(writer, sheet_name = 'Facilities', index = False)




In [182]:
write_excel_file(cdata, full_path, full_path2)

Writing data to file...
Writing data to file...
Markey_catchment_data_01-10-2023.xlsx created


In [ ]:
    def save_as_xlsx():
        from pandas import ExcelWriter
        from datetime import datetime as dt
        
        ca_dir = ca_name.replace(" ", "_") + "_catchment_data"
        path2 = os.path.join(os.getcwd(), ca_dir)
        
        if os.path.exists(path2) == False:
            os.makedirs(path2)
            
        save_name = ca_name.replace(" ", "_") + '_catchment_data_' + dt.today().strftime('%m-%d-%Y') + '.xlsx'
        save_name2 = ca_name.replace(" ", "_") + '_catchment_data_long_' + dt.today().strftime('%m-%d-%Y') + '.xlsx'
        full_path = os.path.join(os.getcwd(), ca_dir, save_name)
        full_path2 = os.path.join(os.getcwd(), ca_dir, save_name2)
    
        with ExcelWriter(full_path, mode = 'w') as writer:
            print('Writing data to file...')
            pd.read_csv('CIFTools_Documentation.csv', 
                        header = None, encoding = "ISO-8859-1").to_excel(writer, header = None, 
                                                                         sheet_name = 'Variables and Sources', index = False)
            cdata['cancer_incidence'].to_excel(writer, sheet_name = 'Cancer Incidence', index = True)
            cdata['cancer_mortality'].to_excel(writer, sheet_name = 'Cancer Mortality', index = True)
            cdata['economy_county'].to_excel(writer, sheet_name = 'Economy (County)', index = False)
            cdata['economy_tract'].to_excel(writer, sheet_name = 'Economy (Tract)', index = False)
            cdata['environment_county'].to_excel(writer, sheet_name = 'Environment (County)', index = False)
            cdata['environment_tract'].to_excel(writer, sheet_name = 'Environment (Tract)', index = False)
            #cdata['broadband_speeds'].to_excel(writer, sheet_name = 'Broadband Speeds', index = False) #can be too long in some areas
            cdata['ht_county'].to_excel(writer, sheet_name = 'H and T (County)', index = False)
            cdata['ht_tract'].to_excel(writer, sheet_name= 'H and T (Tract)', index = False)
            cdata['rf_and_screening_county'].to_excel(writer, sheet_name= 'RF and Screening (County)', index=True)
            cdata['rf_and_screening_tract'].to_excel(writer, sheet_name= 'RF and Screening (Tract)', index=True)
            cdata['sociodemographics_county'].to_excel(writer, sheet_name = 'Sociodemographic (County)', index = False)
            cdata['sociodemographics_tract'].to_excel(writer, sheet_name = 'Sociodemographic (Tract)', index = False)
            cdata['facilities_and_providers'].to_excel(writer, sheet_name = 'Facilities', index = False)
            
        with ExcelWriter(full_path2, mode = 'w') as writer:
            print('Writing data to file...')
            pd.read_csv('CIFTools_Documentation.csv', 
                        header = None, encoding = "ISO-8859-1").to_excel(writer, header = None, 
                                                                         sheet_name = 'Variables and Sources', index = False)
            cdata['cancer_incidence_long'].to_excel(writer, sheet_name = 'Cancer Incidence', index = True)
            cdata['cancer_mortality_long'].to_excel(writer, sheet_name = 'Cancer Mortality', index = True)
            cdata['economy_county_long'].to_excel(writer, sheet_name = 'Economy (County)', index = False)
            cdata['economy_tract_long'].to_excel(writer, sheet_name = 'Economy (Tract)', index = False)
            cdata['environment_county_long'].to_excel(writer, sheet_name = 'Environment (County)', index = False)
            cdata['environment_tract_long'].to_excel(writer, sheet_name = 'Environment (Tract)', index = False)
            #cdata['broadband_speeds'].to_excel(writer, sheet_name = 'Broadband Speeds', index = False) # can be too long in some areas
            cdata['ht_county_long'].to_excel(writer, sheet_name = 'H and T (County)', index = False)
            cdata['ht_tract_long'].to_excel(writer, sheet_name= 'H and T (Tract)', index = False)
            cdata['rf_and_screening_county_long'].to_excel(writer, sheet_name= 'RF and Screening (County)', 
                                                            index=True)
            cdata['rf_and_screening_tract_long'].to_excel(writer, sheet_name= 'RF and Screening (Tract)', 
                                                          index=True)
            cdata['sd_county_long'].to_excel(writer, sheet_name = 'Sociodemographic (County)', index = False)
            cdata['sd_tract_long'].to_excel(writer, sheet_name = 'Sociodemographic (Tract)', index = False)
            cdata['facilities_and_providers'].to_excel(writer, sheet_name = 'Facilities', index = False)
        
        print(save_name + ' created')
        
        return
